In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import pandas as pd
import re as re
import matplotlib.pyplot as plt
import xgboost as xgb

In [3]:
def myPrint(x):
    print(x.head(15))
    print(x.info())

In [5]:
test_b_wa = pd.read_table('../data/Test-B/wa_test_b.txt', header = None, names = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', 'up_flow', 'down_flow', 'wa_type', 'date'])

In [7]:
myPrint(test_b_wa)

      uid  wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  date
0   u7000  139上网导航        1.0        75.0     60.0       76.0      0.0  10.0
1   u7000  139上网导航        2.0      1001.0   2058.0     1131.0      0.0   7.0
2   u7000  139上网导航        3.0      1020.0   1721.0      615.0      0.0  20.0
3   u7000  139上网导航        3.0       370.0   2782.0    12065.0      0.0  23.0
4   u7000  139上网导航        5.0       711.0   1769.0      739.0      0.0   2.0
5   u7000  139上网导航        5.0      8934.0   5933.0    22796.0      0.0   1.0
6   u7000  139上网导航        6.0      9228.0   6819.0    15712.0      0.0  26.0
7   u7000  139上网导航        7.0      8285.0   9316.0    18325.0      0.0   3.0
8   u7000  139上网导航        8.0     16637.0  15256.0    17954.0      0.0  11.0
9   u7000  139上网导航        8.0      7858.0   8869.0     3364.0      0.0  17.0
10  u7000  139上网导航       10.0     12688.0  33456.0   629756.0      0.0  18.0
11  u7000  139上网导航       14.0     19113.0  13871.0    27719.0      0.0  21.0

In [8]:
test_b_voice = pd.read_table('../data/Test-B/voice_test_b.txt', header = None, names = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'end_time', 'call_type', 'in_out'], dtype = {'opp_num':str})

In [9]:
myPrint(test_b_voice)

      uid                           opp_num  opp_head  opp_len  start_time  \
0   u8925  F5FCD87EA7AF344FF408D4A7842CAEF8       187       11    12122145   
1   u8925  06AE2E5A890E9407AD6C0486406B1FD8       159       11    12114240   
2   u8884  E97935F68B5C09034E1259A6A4729D93       183       11    12150612   
3   u8884  CF8091C4559FEE3D78B5EC86617085EF       183       11    12144742   
4   u8884  CADA2EEDD787DF9F882BA75D840FA5F6       136       11    12144405   
5   u8884  CADA2EEDD787DF9F882BA75D840FA5F6       136       11    12145820   
6   u8884  0B5AD1A584FCB2E418CA7658C40C3F9E       138       11    12145405   
7   u8884  CF8091C4559FEE3D78B5EC86617085EF       183       11    12150840   
8   u8884  CADA2EEDD787DF9F882BA75D840FA5F6       136       11    12145531   
9   u8884  B40F3D84D6D48839CD9C41A685EC3A5E       138       11    12151209   
10  u8884  CF8091C4559FEE3D78B5EC86617085EF       183       11    12145246   
11  u8884  5436B0685B7D39EA42BE1977C74983B6       133       11  

In [10]:
test_b_sms = pd.read_table('../data/Test-B/sms_test_b.txt', header = None, names = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'in_out'], dtype = {'opp_num':str})

In [11]:
myPrint(test_b_sms)

      uid                           opp_num  opp_head  opp_len  start_time  \
0   u8774  2475F8BAE0386F837075F1F391732DAE       131       11     3084029   
1   u9971  2D3AB85C703AC8EF8642CFC8DF5F01DE       156       11     3202228   
2   u9971  2D3AB85C703AC8EF8642CFC8DF5F01DE       156       11     3200003   
3   u9971  2D3AB85C703AC8EF8642CFC8DF5F01DE       156       11     3101349   
4   u9971  2D3AB85C703AC8EF8642CFC8DF5F01DE       156       11     3100814   
5   u9771  9783C3AAB5AD009D3F6EBCD4542FE918       132       11     3135825   
6   u9759  FA43C66CCD3425FCF1ED66DEC1F1C3E9       132       11     3101546   
7   u9759  FA43C66CCD3425FCF1ED66DEC1F1C3E9       132       11     3101733   
8   u9759  FA43C66CCD3425FCF1ED66DEC1F1C3E9       132       11     3100521   
9   u9759  FA43C66CCD3425FCF1ED66DEC1F1C3E9       132       11     3100521   
10  u9759  FA43C66CCD3425FCF1ED66DEC1F1C3E9       132       11     3103543   
11  u9759  FA43C66CCD3425FCF1ED66DEC1F1C3E9       132       11  

In [12]:
test_b_wa[test_b_wa['wa_name'].isnull()]

,uid,wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_type,date
1226,u7001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14536,u7021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37236,u7048,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37262,u7051,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37342,u7053,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39449,u7059,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52275,u7077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59789,u7092,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62347,u7096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65137,u7102,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
test_b_wa_df = test_b_wa[True^test_b_wa['wa_name'].isnull()]

In [14]:
myPrint(test_b_wa_df)

      uid  wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  date
0   u7000  139上网导航        1.0        75.0     60.0       76.0      0.0  10.0
1   u7000  139上网导航        2.0      1001.0   2058.0     1131.0      0.0   7.0
2   u7000  139上网导航        3.0      1020.0   1721.0      615.0      0.0  20.0
3   u7000  139上网导航        3.0       370.0   2782.0    12065.0      0.0  23.0
4   u7000  139上网导航        5.0       711.0   1769.0      739.0      0.0   2.0
5   u7000  139上网导航        5.0      8934.0   5933.0    22796.0      0.0   1.0
6   u7000  139上网导航        6.0      9228.0   6819.0    15712.0      0.0  26.0
7   u7000  139上网导航        7.0      8285.0   9316.0    18325.0      0.0   3.0
8   u7000  139上网导航        8.0     16637.0  15256.0    17954.0      0.0  11.0
9   u7000  139上网导航        8.0      7858.0   8869.0     3364.0      0.0  17.0
10  u7000  139上网导航       10.0     12688.0  33456.0   629756.0      0.0  18.0
11  u7000  139上网导航       14.0     19113.0  13871.0    27719.0      0.0  21.0

In [20]:
test_b_wa_df.loc[:,'date'] = test_b_wa_df['date'].astype('int64')

In [22]:
test_b_wa_df.loc[:,['up_flow', 'down_flow']] = test_b_wa_df[['up_flow','down_flow']].astype('int64')

In [23]:
test_b_wa_df.loc[:,['visit_dura', 'wa_type']] = test_b_wa_df[['visit_dura', 'wa_type']].astype('int64')

In [24]:
test_b_wa_df.loc[:,'visit_cnt'] = test_b_wa_df['visit_cnt'].astype('int64')

In [25]:
myPrint(test_b_wa_df)

      uid  wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  date
0   u7000  139上网导航          1          75       60         76        0    10
1   u7000  139上网导航          2        1001     2058       1131        0     7
2   u7000  139上网导航          3        1020     1721        615        0    20
3   u7000  139上网导航          3         370     2782      12065        0    23
4   u7000  139上网导航          5         711     1769        739        0     2
5   u7000  139上网导航          5        8934     5933      22796        0     1
6   u7000  139上网导航          6        9228     6819      15712        0    26
7   u7000  139上网导航          7        8285     9316      18325        0     3
8   u7000  139上网导航          8       16637    15256      17954        0    11
9   u7000  139上网导航          8        7858     8869       3364        0    17
10  u7000  139上网导航         10       12688    33456     629756        0    18
11  u7000  139上网导航         14       19113    13871      27719        0    21

# Voice

In [26]:
test_b_voice_df = test_b_voice

In [27]:
myPrint(test_b_voice_df)

      uid                           opp_num  opp_head  opp_len  start_time  \
0   u8925  F5FCD87EA7AF344FF408D4A7842CAEF8       187       11    12122145   
1   u8925  06AE2E5A890E9407AD6C0486406B1FD8       159       11    12114240   
2   u8884  E97935F68B5C09034E1259A6A4729D93       183       11    12150612   
3   u8884  CF8091C4559FEE3D78B5EC86617085EF       183       11    12144742   
4   u8884  CADA2EEDD787DF9F882BA75D840FA5F6       136       11    12144405   
5   u8884  CADA2EEDD787DF9F882BA75D840FA5F6       136       11    12145820   
6   u8884  0B5AD1A584FCB2E418CA7658C40C3F9E       138       11    12145405   
7   u8884  CF8091C4559FEE3D78B5EC86617085EF       183       11    12150840   
8   u8884  CADA2EEDD787DF9F882BA75D840FA5F6       136       11    12145531   
9   u8884  B40F3D84D6D48839CD9C41A685EC3A5E       138       11    12151209   
10  u8884  CF8091C4559FEE3D78B5EC86617085EF       183       11    12145246   
11  u8884  5436B0685B7D39EA42BE1977C74983B6       133       11  

# sms

In [28]:
test_b_sms_df = test_b_sms

In [29]:
def do2(x):
    x = int(x.replace('u', ''))
    return x
def do(dataset):
    ans = dataset
    ans.loc[:, 'uid'] = ans['uid'].apply(do2)
    
    return ans

In [30]:
test_b_voice_df_2 = do(test_b_voice_df)

In [31]:
test_b_sms_df_2 = do(test_b_sms_df)

In [32]:
test_b_wa_df_2 = do(test_b_wa_df)

In [33]:
myPrint(test_b_wa_df_2)

     uid  wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  date
0   7000  139上网导航          1          75       60         76        0    10
1   7000  139上网导航          2        1001     2058       1131        0     7
2   7000  139上网导航          3        1020     1721        615        0    20
3   7000  139上网导航          3         370     2782      12065        0    23
4   7000  139上网导航          5         711     1769        739        0     2
5   7000  139上网导航          5        8934     5933      22796        0     1
6   7000  139上网导航          6        9228     6819      15712        0    26
7   7000  139上网导航          7        8285     9316      18325        0     3
8   7000  139上网导航          8       16637    15256      17954        0    11
9   7000  139上网导航          8        7858     8869       3364        0    17
10  7000  139上网导航         10       12688    33456     629756        0    18
11  7000  139上网导航         14       19113    13871      27719        0    21
12  7000  13

In [34]:
test_b_voice_df_2.to_csv('../data/Test-B/test_b_voice_df.csv', index=False)
test_b_sms_df_2.to_csv('../data/Test-B/test_b_sms_df.csv', index=False)
test_b_wa_df_2.to_csv('../data/Test-B/test_b_wa_df.csv', index=False)